In [3]:
#코사인 유사도
#유사도를 이용한 추천 시스템 구현하기

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data=pd.read_csv(r"C:\Users\Msi\Desktop\movies_metadata.csv",low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
#코사인 유사도에 사용할 데이터는 영화제목에 해당하는 title열과 줄거리에 해당하는 overview열이다. 
#좋아하는 영화를 입력하면 해당 영화의 줄거리와 줄거리가 유사한 영화를 찾아서 추천하는 시스템을 만들자

data=data.head(20000)#20000개만 한번 해보자!
data['overview'].isnull().sum()
#135개의 샘플에서 Null값이 있다. 

135

In [5]:
data['overview']=data['overview'].fillna('') #널값을 제거한다.
tfidf=TfidfVectorizer(stop_words='english')
tfidf_matrix=tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)#overview열에 대해서 tf-idf를 수행.
#20000개의 영화를 표현하기 위해 47487개의 단어 사용

(20000, 47487)


In [6]:
cosine_sim=linear_kernel(tfidf_matrix,tfidf_matrix) #코사인 유사도 구하기

In [7]:
indices=pd.Series(data.index,index=data['title']).drop_duplicates()
print(indices.head()) #5개만 출력해보도록 하자!

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64


In [9]:
idx=indices['Father of the Bride Part II']
print(idx)

4


In [11]:
def get_recommendations(title,cosine_sim=cosine_sim):
    #선택한 영화의 타이틀로부터 해당되는 인덱스를 받아온다. 선택한 영화를 가지고 연산할 수 있다.
    idx=indices[title]
    #모든 영화에 대해서 해당 영화와의 유사도를 구한다.
    sim_scores=list(enumerate(cosine_sim[idx]))
    #유사도에 따라 영화들을 정렬한다.
    sim_scores=sorted(sim_scores,key=lambda x : x[1],reverse=True)
    #가장 유사한 10개의 영화를 받아온다.
    sim_scores=sim_scores[1:11]
    #가장 유사한 10개의 영화의 인덱스를 받아온다.
    movie_indices=[i[0] for i in sim_scores]
    #가장 유사한 10개의 영화 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

In [12]:
#여러가지 유사도 기법
#자카드 유사도
#합집합에서 교집합의 비율을 구한다면 두 집합 A와 B의 유사도를 구할 수 있다.
#자카드 함수 유사도 구하기

#두 문서 모두에서 등장한 단어는 apple과 banana 2개
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

#토큰화 수행한다.
tokenized_doc1=doc1.split()
tokenized_doc2=doc2.split()
print(tokenized_doc1)
print(tokenized_doc2)

['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [13]:
union=set(tokenized_doc1).union(set(tokenized_doc2))
print(union)
#합집합 만들기

{'likey', 'banana', 'card', 'you', 'apple', 'holder', 'passport', 'watch', 'coupon', 'everyone', 'like', 'love'}


In [14]:
#교집합 만들기
intersection=set(tokenized_doc1).intersection(set(tokenized_doc2))
print(intersection)

{'banana', 'apple'}


In [15]:
print(len(intersection)/len(union))

0.16666666666666666
